In [16]:
import geopandas as gpd
import folium
import leafmap.leafmap as leafmap


In [17]:
region = 'bharatpur'
meta_data_path = f"../data/region_performance/{region}/metadata.geojson"
region_meta_data = gpd.read_file(meta_data_path)


display(region_meta_data.head(2))


,x_idx,y_idx,x,y,geometry
0,6080,24512,8580211,3111596,"POLYGON ((8578682.946 3113124.899, 8581740.427..."
1,7232,25088,8585715,3108844,"POLYGON ((8584186.412 3110373.166, 8587243.893..."


In [18]:
region_meta_data = region_meta_data.to_crs("EPSG:3857")
display(region_meta_data.head(2))

,x_idx,y_idx,x,y,geometry
0,6080,24512,8580211,3111596,"POLYGON ((8578682.946 3113124.899, 8581740.427..."
1,7232,25088,8585715,3108844,"POLYGON ((8584186.412 3110373.166, 8587243.893..."


In [19]:
#print the first geometry
first_geometry = region_meta_data.iloc[[0]]
m = leafmap.Map()
m.add_basemap("HYBRID")
m.add_gdf(region_meta_data, layer_name="Shape", style={"color": "black"},zoom_to_layer=15)
m.add_gdf(first_geometry,zoom_to_layer=True)


m.add_gdf(first_geometry, layer_name="First Geometry", style={"color": "red"}, zoom_to_layer=True)
m




Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [20]:
from os.path import join
from glob import glob
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
from time import time
from joblib import Parallel, delayed
from tqdm import tqdm

In [21]:
# region = 'alwar'
image_dir=f"../data/region_performance/{region}/images"
prediction_dir=f"../predict/m1_obb_without_empty_train_obb_v1_640_100/{region}/"
metadata_path=f"../data/region_performance/{region}/metadata.geojson"
save_dir=f"../predict/m1_obb_without_empty_train_obb_v1_640_100/{region}/"

print(f"Image dir: {image_dir}")
print(f"Prediction dir: {prediction_dir}")
print(f"Metadata path: {metadata_path}")




Image dir: ../data/region_performance/bharatpur/images
Prediction dir: ../predict/m1_obb_without_empty_train_obb_v1_640_100/bharatpur/
Metadata path: ../data/region_performance/bharatpur/metadata.geojson


In [22]:
print(f"Image directory: {image_dir}")
print(f"Prediction directory: {prediction_dir}")

image_paths = glob(join(image_dir, "*"))
print(f"Number of images: {len(image_paths)}")
prediction_path = join(prediction_dir, "labels", "*")
print("Reading predictions from", prediction_path)
prediction_paths = glob(prediction_path)
print(f"Number of predictions: {len(prediction_paths)}")
print(f"Number of images with at least one prediction: {len(prediction_paths)}")
metadata_gdf = gpd.read_file(metadata_path)
# print(f"Number of metadata entries: {len(metadata_gdf)}")
# target_gdf = gpd.read_file(target_geojson_path)
# print(f"Number of ground truth labels: {len(target_gdf)}")
# print(f"Time taken to read files: {time() - init:.2f}s")

Image directory: ../data/region_performance/bharatpur/images
Prediction directory: ../predict/m1_obb_without_empty_train_obb_v1_640_100/bharatpur/
Number of images: 686
Reading predictions from ../predict/m1_obb_without_empty_train_obb_v1_640_100/bharatpur/labels/*
Number of predictions: 604
Number of images with at least one prediction: 604


In [23]:
print("Predictions head")
print(metadata_gdf.head(2))

Predictions head
   x_idx  y_idx        x        y  \
0   6080  24512  8580211  3111596   
1   7232  25088  8585715  3108844   

                                            geometry  
0  POLYGON ((8578682.946 3113124.899, 8581740.427...  
1  POLYGON ((8584186.412 3110373.166, 8587243.893...  


In [24]:
classes = ["CFCBK", "FCBK", "Zigzag"]
conf_threshold=0.25
nms_iou=0.5
task="obb"
def obb_load_prediction(row,task=task,classes=classes):
    try:
        prediction = np.loadtxt(join(prediction_dir, "labels", f"{row['x']}_{row['y']}.txt"), ndmin=2)
    except FileNotFoundError:
        prediction = np.zeros((0, 10)) if task == "obb" else np.zeros((0, 6))

    # Preserve original prediction for later
    original_prediction = prediction.copy().tolist()
    original_prediction = ["_".join(map(str, x)) for x in original_prediction]

    # scale predictions
    min_x, min_y, max_x, max_y = row["geometry"].bounds
    prediction[:, 1:-1:2] = prediction[:, 1:-1:2] * (max_x - min_x) + min_x
    prediction[:, 2:-1:2] = (1 - prediction[:, 2:-1:2]) * (max_y - min_y) + min_y
    class_names = [classes[int(cls_id)] for cls_id in prediction[:, 0]]
    confidence = prediction[:, -1].tolist()

    box = prediction[:, 1:-1]
    return box, class_names, confidence, original_prediction



In [25]:
init = time()
metadata_gdf[["box", "class_name", "confidence", "yolo_label"]] = metadata_gdf.apply(
    obb_load_prediction, axis=1, result_type="expand"
)
print("Predictions head after getting box, class_name, confidence")
print(metadata_gdf.head(2))
print(f"Time taken to load predictions: {time() - init:.2f}s")

init = time()
print("Length before explode: ", len(metadata_gdf))
metadata_gdf = metadata_gdf.apply(pd.Series.explode).reset_index(drop=True)
print("Length after explode: ", len(metadata_gdf))
metadata_gdf = metadata_gdf.dropna(subset=["box"]).reset_index(drop=True)
print("Length after dropping NaN: ", len(metadata_gdf))
print(metadata_gdf.head(2))
print(f"Time taken to explode predictions: {time() - init:.2f}s")

print("Length before conf filtering: ", len(metadata_gdf))
metadata_gdf = metadata_gdf[metadata_gdf["confidence"] >= float(conf_threshold)]
print("Length after conf filtering: ", len(metadata_gdf))

init = time()

Predictions head after getting box, class_name, confidence
   x_idx  y_idx        x        y  \
0   6080  24512  8580211  3111596   
1   7232  25088  8585715  3108844   

                                            geometry  \
0  POLYGON ((8578682.946 3113124.899, 8581740.427...   
1  POLYGON ((8584186.412 3110373.166, 8587243.893...   

                                                 box            class_name  \
0  [[8580201.226896087, 3110813.1496623135, 85802...  [Zigzag, FCBK, FCBK]   
1  [[8584186.1875373, 3108303.611954807, 8584219....    [FCBK, FCBK, FCBK]   

                             confidence  \
0  [0.00399701, 0.00167417, 0.00110212]   
1   [0.00209298, 0.0017719, 0.00150438]   

                                          yolo_label  
0  [2.0_0.496579_0.756096_0.500575_0.742218_0.470...  
1  [1.0_-7.34173e-05_0.676882_0.0108705_0.678477_...  
Time taken to load predictions: 0.14s
Length before explode:  686
Length after explode:  1916
Length after dropping NaN:  1834
   

In [26]:
init = time()
if task == "obb":
    #     # metadata_gdf["label_geometry"] = metadata_gdf["box"].apply(
    #     #     lambda box: Polygon(
    #     #         [
    #     #             (box[0], box[1]),
    #     #             (box[2], box[3]),
    #     #             (box[4], box[5]),
    #     #             (box[6], box[7]),
    #     #             (box[0], box[1]),
    #     #         ]
    #     #     )
    #     # )
    # metadata_gdf["label_geometry"] = np.apply_along_axis(
    #     lambda x: Polygon(x.reshape(-1, 2)), 1, np.asarray(metadata_gdf.box.tolist())
    # )
    # metadata_gdf["label_geometry"] = Parallel(n_jobs=32)(
    #     delayed(Polygon)(box.reshape(-1, 2)) for box in tqdm(metadata_gdf.box)
    # )
    metadata_gdf["label_geometry"] = metadata_gdf["box"].apply(lambda x: Polygon(x.reshape(-1, 2)))
    # metadata_gdf["label_geometry"] = metadata_gdf.box.apply(
    #     lambda x: {"type": "Polygon", "coordinates": [x.reshape(-1, 2).tolist()]}
    # )
# else:
#     metadata_gdf["label_geometry"] = metadata_gdf["box"].apply(
#         lambda box: Polygon(
#             [
#                 (box[0] - box[2] / 2, box[1] - box[3] / 2),
#                 (box[0] + box[2] / 2, box[1] - box[3] / 2),
#                 (box[0] + box[2] / 2, box[1] + box[3] / 2),
#                 (box[0] - box[2] / 2, box[1] + box[3] / 2),
#                 (box[0] - box[2] / 2, box[1] - box[3] / 2),
#             ]
#         )
#     )
print(metadata_gdf.head(2))
print(f"Time taken to convert predictions to geometry: {time() - init:.2f}s")

crs = metadata_gdf.crs
metadata_gdf.drop(columns=["box", "x_idx", "y_idx", "geometry"], inplace=True)
metadata_gdf.rename(columns={"label_geometry": "geometry"}, inplace=True)
metadata_gdf.set_geometry("geometry", inplace=True)
metadata_gdf.crs = crs

############# Overlap removal
metadata_gdf.reset_index(drop=True, inplace=True)
print(f"{type(metadata_gdf)=}")
print(f"{metadata_gdf.columns=}")
print(f"{metadata_gdf.crs=}")
intersection_gdf = gpd.sjoin(metadata_gdf, metadata_gdf, predicate="intersects")
# remove same points and duplicate pairs
intersection_gdf = intersection_gdf[intersection_gdf.index < intersection_gdf.index_right][
    ["index_right"]
].reset_index(drop=False)
intersection_gdf.rename(columns={"index": "index_left"}, inplace=True)

def get_iou(row):
    geometry_left = metadata_gdf.loc[row.index_left, "geometry"]
    geometry_right = metadata_gdf.loc[row.index_right, "geometry"]
    return geometry_left.intersection(geometry_right).area / geometry_left.union(geometry_right).area

intersection_gdf["iou"] = intersection_gdf.apply(get_iou, axis=1)

def get_remove_indices(row):
    if row.iou >= nms_iou:
        left_area = metadata_gdf.loc[row.index_left, "geometry"].area
        right_area = metadata_gdf.loc[row.index_right, "geometry"].area
        return row.index_left if left_area > right_area else row.index_right

intersection_gdf["index_remove"] = intersection_gdf.apply(get_remove_indices, axis=1)
intersection_gdf.dropna(subset=["index_remove"], inplace=True)
print("Size before NMS: ", len(metadata_gdf))
metadata_gdf.drop(index=intersection_gdf["index_remove"], inplace=True)
print("Size after NMS: ", len(metadata_gdf))

print(f"{metadata_gdf.class_name.isnull().sum()=}")

metadata_gdf["confidence"] = metadata_gdf["confidence"].astype(float)
print(f"{metadata_gdf.dtypes=}")

init = time()
metadata_gdf.to_file(
    join(save_dir, f"predictions_{conf_threshold}.geojson"),
    driver="GeoJSON",
)
print(f"Time taken to save predictions: {time() - init:.2f}s")
# print(f"Total time taken: {time() - overall_init:.2f}s")

     x_idx  y_idx        x        y  \
134   9536  25088  8596722  3108844   
325   7232  18752  8585715  3139113   

                                              geometry  \
134  POLYGON ((8595193.344 3110373.166, 8598250.825...   
325  POLYGON ((8584186.412 3140642.229, 8587243.893...   

                                                   box class_name confidence  \
134  [8598173.406732827, 3108868.267570076, 8598211...       FCBK   0.482616   
325  [8586812.100367565, 3138632.724080379, 8586815...      CFCBK   0.863236   

                                            yolo_label  \
134  1.0_0.974679_0.492202_0.987097_0.505247_1.0124...   
325  0.0_0.858775_0.657242_0.85979_0.627291_0.82920...   

                                        label_geometry  
134  POLYGON ((8598173.406732827 3108868.267570076,...  
325  POLYGON ((8586812.100367565 3138632.724080379,...  
Time taken to convert predictions to geometry: 0.01s
type(metadata_gdf)=<class 'geopandas.geodataframe.GeoDataFrame'>
me

In [27]:
print("Predictions head after saving")
print(metadata_gdf.head(2))
# print total number of predictions
print(f"Number of predictions: {len(metadata_gdf)}")

Predictions head after saving
         x        y class_name  confidence  \
0  8596722  3108844       FCBK    0.482616   
1  8585715  3139113      CFCBK    0.863236   

                                          yolo_label  \
0  1.0_0.974679_0.492202_0.987097_0.505247_1.0124...   
1  0.0_0.858775_0.657242_0.85979_0.627291_0.82920...   

                                            geometry  
0  POLYGON ((8598173.407 3108868.268, 8598211.375...  
1  POLYGON ((8586812.1 3138632.724, 8586815.204 3...  
Number of predictions: 101
